In [1]:
import numpy as np

num_features = 1
loc = 1

x_train = np.vstack([np.random.normal(size=(9000, num_features)), np.random.normal(loc=loc, size=(1000, num_features))])
x_train.shape

(10000, 1)

In [2]:
y_train = np.hstack([np.zeros((9000,)), np.ones((1000,))])
y_train.shape

(10000,)

In [3]:
x_test = np.vstack([np.random.normal(size=(900, num_features)), np.random.normal(loc=loc, size=(100, num_features))])
x_test.shape

(1000, 1)

In [4]:
y_test = np.hstack([np.zeros((900,)), np.ones((100,))])
y_test.shape

(1000,)

In [5]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import keras.backend as K

def auc(y_true, y_pred):
    y_true_1 = tf.equal(y_true, 1)
    y_true_0 = tf.equal(y_true, 0)
    a = tf.boolean_mask(y_pred, y_true_1)
    b = tf.boolean_mask(y_pred, y_true_0)
    return K.mean(a[None, :] > b[:, None])

Using TensorFlow backend.


In [6]:
model = Sequential()
#model.add(Dense(100, input_shape=(num_features,), activation='relu'))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(100, activation='relu'))
model.add(Dense(1, input_shape=(num_features,), activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])

In [7]:
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_data=(x_test, y_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/10
10000/10000 [==============================] - 1s - loss: 0.8556 - acc: 0.5432 - auc: 0.7675 - val_loss: 0.8038 - val_acc: 0.5600 - val_auc: nan
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 0.7929 - acc: 0.5687 - auc: 0.7723 - val_loss: 0.7452 - val_acc: 0.5870 - val_auc: nan
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 0.7351 - acc: 0.5971 - auc: 0.7691 - val_loss: 0.6910 - val_acc: 0.6120 - val_auc: nan
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 0.6819 - acc: 0.6231 - auc: 0.7669 - val_loss: 0.6418 - val_acc: 0.6360 - val_auc: nan
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 0.6335 - acc: 0.6519 - auc: 0.7706 - val_loss: 0.5971 - val_acc: 0.6700 - val_auc: nan
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 0.5897 - acc: 0.6819 - auc: 0.7698 - val_loss: 0.5567 - val_acc: 0.6990 - val_auc: nan
Epoch 7/10
1000

In [8]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)
confusion_matrix(y_pred > 0.5, y_test)

array([[783,  53],
       [117,  47]])

In [9]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_pred)

0.78566666666666674

In [10]:
model = Sequential()
#model.add(Dense(100, input_shape=(num_features,), activation='relu'))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(100, activation='relu'))
model.add(Dense(1, input_shape=(num_features,), activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])

In [11]:
from sklearn.utils.class_weight import compute_class_weight

class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight

array([ 0.55555556,  5.        ])

In [12]:
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_data=(x_test, y_test), class_weight=class_weight)

Train on 10000 samples, validate on 1000 samples
Epoch 1/10
10000/10000 [==============================] - 0s - loss: 0.9343 - acc: 0.4788 - auc: 0.2290 - val_loss: 0.9208 - val_acc: 0.4790 - val_auc: nan
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 0.8694 - acc: 0.5070 - auc: 0.2288 - val_loss: 0.8559 - val_acc: 0.5060 - val_auc: nan
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 0.8088 - acc: 0.5383 - auc: 0.2279 - val_loss: 0.7956 - val_acc: 0.5400 - val_auc: nan
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 0.7527 - acc: 0.5755 - auc: 0.2262 - val_loss: 0.7401 - val_acc: 0.5780 - val_auc: nan
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 0.7013 - acc: 0.6169 - auc: 0.2353 - val_loss: 0.6893 - val_acc: 0.6160 - val_auc: nan
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 0.6544 - acc: 0.6644 - auc: 0.2270 - val_loss: 0.6432 - val_acc: 0.6650 - val_auc: nan
Epoch 7/10
1000

In [13]:
y_pred = model.predict(x_test)
confusion_matrix(y_pred > 0.5, y_test)

array([[874, 100],
       [ 26,   0]])

In [14]:
roc_auc_score(y_test, y_pred)

0.21433333333333332

In [15]:
model = Sequential()
#model.add(Dense(100, input_shape=(num_features,), activation='relu'))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(100, activation='relu'))
model.add(Dense(1, input_shape=(num_features,), activation='sigmoid'))

from utils import sample_binomial
from keras.losses import binary_crossentropy

def reinforce_loss(y_true, y_pred):
    y_samp = sample_binomial(y_pred)
    rewards = auc(y_true, y_samp)
    baseline = auc(y_true, y_pred)
    advantages = rewards - baseline
    return advantages * binary_crossentropy(y_samp, y_pred)

model.compile(loss=reinforce_loss, optimizer='adam', metrics=['accuracy', auc])

In [16]:
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_data=(x_test, y_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/10
10000/10000 [==============================] - 0s - loss: -0.0502 - acc: 0.4673 - auc: 0.2265 - val_loss: nan - val_acc: 0.4570 - val_auc: nan
Epoch 2/10
10000/10000 [==============================] - 0s - loss: -0.0503 - acc: 0.4653 - auc: 0.2302 - val_loss: nan - val_acc: 0.4580 - val_auc: nan
Epoch 3/10
10000/10000 [==============================] - 0s - loss: -0.0586 - acc: 0.4692 - auc: 0.2335 - val_loss: nan - val_acc: 0.4610 - val_auc: nan
Epoch 4/10
10000/10000 [==============================] - 0s - loss: -0.0440 - acc: 0.4704 - auc: 0.2323 - val_loss: nan - val_acc: 0.4580 - val_auc: nan
Epoch 5/10
10000/10000 [==============================] - 0s - loss: -0.0528 - acc: 0.4670 - auc: 0.2282 - val_loss: nan - val_acc: 0.4580 - val_auc: nan
Epoch 6/10
10000/10000 [==============================] - 0s - loss: -0.0480 - acc: 0.4684 - auc: 0.2270 - val_loss: nan - val_acc: 0.4610 - val_auc: nan
Epoch 7/10
10000/10000 [===

In [17]:
y_pred = model.predict(x_test)
confusion_matrix(y_pred > 0.6, y_test)

array([[578,  97],
       [322,   3]])

In [18]:
roc_auc_score(y_test, y_pred)

0.21433333333333332

In [19]:
model = Sequential()
#model.add(Dense(100, input_shape=(num_features,), activation='relu'))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(100, activation='relu'))
model.add(Dense(1, input_shape=(num_features,), activation='sigmoid'))

def auc_loss(y_true, y_pred):
    y_true_1 = tf.equal(y_true[:, 0], 1)
    y_true_0 = tf.equal(y_true[:, 0], 0)
    prob_1 = tf.boolean_mask(y_pred, y_true_1)
    prob_0 = tf.boolean_mask(y_pred, y_true_0)
    agtb = tf.cast(prob_1[:, None] > prob_0[None, :], tf.float32)
    baseline = tf.reduce_mean(agtb)
    rewards_1 = tf.reduce_mean(agtb, axis=1)
    rewards_0 = tf.reduce_mean(agtb, axis=0)
    advantages_1 = rewards_1 - baseline
    advantages_0 = rewards_0 - baseline
    return tf.reduce_mean(advantages_1 * -tf.log(prob_1)) + tf.reduce_mean(advantages_0 * tf.log(prob_0))

model.compile(loss=auc_loss, optimizer='adam', metrics=['accuracy', auc])

In [20]:
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_data=(x_test, y_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/10
 2800/10000 [=======>......................] - ETA: 0s - loss: -0.2093 - acc: 0.5264 - auc: 0.7669

/home/tambet/.conda/envs/robotex/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


10000/10000 [==============================] - 0s - loss: -0.2300 - acc: 0.5440 - auc: 0.7676 - val_loss: nan - val_acc: 0.5600 - val_auc: nan
Epoch 2/10
10000/10000 [==============================] - 0s - loss: -0.2477 - acc: 0.5649 - auc: 0.7739 - val_loss: nan - val_acc: 0.5810 - val_auc: nan
Epoch 3/10
10000/10000 [==============================] - 0s - loss: -0.2706 - acc: 0.5828 - auc: 0.7704 - val_loss: nan - val_acc: 0.5940 - val_auc: nan
Epoch 4/10
10000/10000 [==============================] - 0s - loss: -0.2886 - acc: 0.5996 - auc: 0.7738 - val_loss: nan - val_acc: 0.6090 - val_auc: nan
Epoch 5/10
10000/10000 [==============================] - 0s - loss: -0.3147 - acc: 0.6139 - auc: 0.7698 - val_loss: nan - val_acc: 0.6180 - val_auc: nan
Epoch 6/10
10000/10000 [==============================] - 0s - loss: -0.3394 - acc: 0.6260 - auc: 0.7697 - val_loss: nan - val_acc: 0.6300 - val_auc: nan
Epoch 7/10
10000/10000 [==============================] - 0s - loss: -0.3634 - acc: 0.6

In [21]:
y_pred = model.predict(x_test)
confusion_matrix(y_pred > 0.9, y_test)

array([[821,  63],
       [ 79,  37]])

In [22]:
roc_auc_score(y_test, y_pred)

0.78566666666666674